# Análisis de Contratos Públicos CompraNet (2024)


### Resumen

En este proyecto se utilizó Python para analizar un set de datos en el que se detallan los contratos públicos registrados en el portal CompraNet durante el ejercicio 2024. 

Los datos se utilizaron para responder a las siguientes preguntas:
* ¿Qué instituciones de gobierno fueron las que más dinero otorgaron con contratos públicos?
* ¿Cuáles fueron los proveedores que más dinero recibieron durante el año?
* ¿Qué porcentaje del total del dinero otorgado en contratos públicos corresponde a adjudicaciones directas?
* En promedio, ¿cuánto dinero del total otorgado en contratos públicos corresponde a adjudicaciones directas?
* ¿Qué instituciones de gobierno otorgaron el mayor monto en contratos de adjudicación directa?

### Datos
Los datos fueron extraídos de la página de CompraNet. Se descargó el archivo CSV etiquetado en la página como “Plataforma Integral Compranet 2024”. 

El archivo, junto con otros sets de datos, pueden encontrarse aquí: [https://upcp-compranet.buengobierno.gob.mx/informacion_ayuda/datos_abiertos.html](https://upcp-compranet.buengobierno.gob.mx/informacion_ayuda/datos_abiertos.html)

El archivo CSV original contiene más campos que los mostrados en el df de abajo. Consideramos conveniente trabajar con un df más pequeño, manteniendo los campos más relevantes para este análisis.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.display.float_format = '{:.0f}'.format

contratos_2024 = pd.read_csv(r'C:\Users\kazrc\OneDrive\Desktop\Contratos2022\contratos_2024.csv', encoding='ISO-8859-1')
contratos_2024 = contratos_2024[['Descripción Ramo', 'Institución', 'Nombre de la UC', 'Título del contrato', 'Descripción del contrato', 'Importe DRC', 'Proveedor o contratista', 'rfc', 'Nacionalidad proveedor o contratista', 'Estratificación', 'Tipo Procedimiento', 'Número de procedimiento','Código del contrato']]
contratos_2024.head()

C:\Users\kazrc\AppData\Local\Temp\ipykernel_6176\2889498699.py:6: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  contratos_2024 = pd.read_csv(r'C:\Users\kazrc\OneDrive\Desktop\Contratos2022\contratos_2024.csv', encoding='ISO-8859-1')


,Descripción Ramo,Institución,Nombre de la UC,Título del contrato,Descripción del contrato,Importe DRC,Proveedor o contratista,rfc,Nacionalidad proveedor o contratista,Estratificación,Tipo Procedimiento,Número de procedimiento,Código del contrato
0,DEFENSA NACIONAL,"GRUPO AEROPORTUARIO, FERROVIARIO, DE SERVICIOS...",DIRECCIÓN GENERAL DE RECURSOS MATERIALES Y SER...,ELABORACIÓN E IMPLEMENTACIÓN DE SISTEMA DE GES...,DISEÑAR Y ASESORAR EN LA ELABORACIÓN E IMPLEME...,712218,ANALYSIS & EXPERTISE SC,A&E0509282Q9,MEXICANA,PEQUEÑA,ADJUDICACIÓN DIRECTA POR MONTOS MÁXIMOS POR EX...,AA-07-H0C-007H0C999-N-84-2024,C-2024-00118295
1,HACIENDA Y CRÉDITO PÚBLICO,INSTITUTO DE ADMINISTRACION Y AVALUOS DE BIENE...,DIRECCIÓN GENERAL DE AVALÚOS Y OBRAS,01-24-176 SERVICIOS PROFESIONALES PARA LA EL...,"ATENDER LA SOLICITUD DE SERVICIOS VALUATORIO, ...",10236,ANAYA AMOR ARQUITECTOS SA DE CV,AAA050125HB8,MEXICANA,MICRO,"ADJUDICACIÓN DIRECTA POR PATENTES, LICENCIAS, ...",AA-06-A00-006A00997-N-1838-2024,C-2024-00103558
2,HACIENDA Y CRÉDITO PÚBLICO,INSTITUTO DE ADMINISTRACION Y AVALUOS DE BIENE...,DIRECCIÓN GENERAL DE AVALÚOS Y OBRAS,1-24-179 SERVICIOS PROFESIONALES PARA LA ELABO...,"ATENDER LA SOLICITUD DE SERVICIOS VALUATORIO, ...",20508,ANAYA AMOR ARQUITECTOS SA DE CV,AAA050125HB8,MEXICANA,MICRO,"ADJUDICACIÓN DIRECTA POR PATENTES, LICENCIAS, ...",AA-06-A00-006A00997-N-1960-2024,C-2024-00106413
3,HACIENDA Y CRÉDITO PÚBLICO,PRONOSTICOS PARA LA ASISTENCIA PUBLICA,GERENCIA DE RECURSOS MATERIALES,SERVCIO DE ASESORÍA JURÍDICA ESPECIAIZADA EN M...,SERVCIO DE ASESORÍA JURÍDICA ESPECIAIZADA EN M...,2420000,AMARO AMARO Y ARAGON SC,AAA1607281A4,MEXICANA,PEQUEÑA,ADJUDICACIÓN DIRECTA POR CONSULTORÍA,AA-06-HJY-006HJY001-N-26-2024,C-2024-00025880
4,SALUD,SECRETARIA DE SALUD,DIRECCION GENERAL DE RECURSOS MATERIALES Y SER...,SERVICIOS INTEGRALES DE ENTREGA PUERTA A PUERT...,SERVICIOS INTEGRALES DE ENTREGA PUERTA A PUERT...,203600,AGENCIA ADUANERA DE AMERICA EN AEREO SC,AAA930521IJ4,MEXICANA,MEDIANA,ADJUDICACIÓN DIRECTA POR MONTOS MÁXIMOS POR EX...,AA-12-512-012000991-N-12-2024,C-2024-00023658


También fue necesario modificar los valores en la columna "Tipo Procedimiento". Esta columna etiqueta el tipo de procedimiento de asignación (Licitación pública, Adjudicación directa, Invitación a por lo menos tres personas o Proceso entre entes públicos) para cada contrato. 

Los procesos etiquetados en el df de abajo con los valores "Adjudicación directa" e "Invitación a por lo menos tres personas" estaban marcados con una variedad más amplia de valores. Tanto las adjudicaciones directas como los procesos por invitación contaban con subcategorías que justifican la razón por la que el proceso no fue una licitación pública abierta.

In [3]:
adjudicaciones = ['ADJUDICACIÓN DIRECTA POR MONTOS MÁXIMOS POR EXCEPCIÓN',
'ADJUDICACIÓN DIRECTA POR URGENCIA Y EVENTUALIDAD',
'ADJUDICACIÓN DIRECTA POR COMERCIALIZACIÓN DIRECTA',
'ADJUDICACIÓN DIRECTA POR SERVICIOS PRESTADOS POR PERSONA FÍSICA, QUE NO REQUIERA DE OTRO ESPECIALISTA O TÉCNICO',
'ADJUDICACIÓN DIRECTA POR CASO FORTUITO O FUERZA MAYOR',
'ADJUDICACIÓN DIRECTA POR PATENTES, LICENCIAS, OFERENTE ÚNICO U OBRAS DE ARTE',
'ADJUDICACIÓN DIRECTA POR ADJUDICACIÓN A PROVEEDOR CON CONTRATO VIGENTE, BAJO LAS MISMAS CONDICIONES',
'ADJUDICACIÓN DIRECTA POR LICITACIONES PÚBLICAS DESIERTAS',
'ADJUDICACIÓN DIRECTA POR INEXISTENCIA DE MARCAS ALTERNATIVAS',
'ADJUDICACIÓN DIRECTA POR AUTORIZACIÓN DE OM O EQUIVALENTE',
'ADJUDICACIÓN DIRECTA POR INVITACIÓN A CUANDO MENOS TRES PERSONAS DESIERTA',
'ADJUDICACIÓN DIRECTA PARA MATERIALES DE USO EN ACTIVIDADES EXPERIMENTALES',
'ADJUDICACIÓN DIRECTA POR CONTRATO MARCO',
'ADJUDICACIÓN DIRECTA POR MANTENIMIENTO SIN PRECISIÓN DEL ALCANCE',
'ADJUDICACIÓN DIRECTA POR BIENES PERECEDEROS O USADOS',
'ADJUDICACIÓN DIRECTA POR PERDIDAS ADICIONALES JUSTIFICADAS',
'ADJUDICACIÓN DIRECTA POR CONTRATACIÓN CON CAMPESINOS O GRUPOS URBANOS',
'ADJUDICACIÓN DIRECTA POR CONSULTORÍA',
'ADJUDICACIÓN DIRECTA POR TRABAJOS EXTRAORDINARIOS',
'ADJUDICACIÓN DIRECTA POR CONTRATOS RESCINDIDOS',
'ADJUDICACIÓN DIRECTA POR DACIÓN DE PAGO',
'ADJUDICACIÓN DIRECTA POR BIENES EN LIQUIDACIÓN',
'ADJUDICACIÓN DIRECTA POR DISEÑO Y FABRICACIÓN DE PROTOTIPOS',
'ADJUDICACIÓN DIRECTA POR CRÉDITO EXTERNO',
'ADJUDICACIÓN DIRECTA POR OPERACIÓN DE INSTALACIONES NUCLEARES',
'ADJUDICACIÓN DIRECTA POR ESTUDIOS PREVIOS A LA LICITACIÓN']

invitaciones = ['INVITACIÓN A CUANDO MENOS 3 PERSONAS POR MONTOS MÁXIMOS POR EXCEPCIÓN',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR MONTOS MÁXIMOS PARA EXCEPCIÓN',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR CONTRATO MARCO',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR LICITACIONES PÚBLICAS DESIERTAS',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR ADJUDICACIÓN A PROVEEDOR CON CONTRATO VIGENTE, BAJO LAS MISMAS CONDICIONES',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR URGENCIA Y EVENTUALIDAD',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR CASO FORTUITO O FUERZA MAYOR',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR INEXISTENCIA DE MARCAS ALTERNATIVAS',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR CONSULTORÍA',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR BIENES PERECEDEROS O USADOS',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR MATERIALES DE USO EN ACTIVIDADES EXPERIMENTALES',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR MANTENIMIENTO SIN PRECISIÓN DEL ALCANCE',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR PERDIDAS ADICIONALES JUSTIFICADAS',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR PATENTES, LICENCIAS, OFERENTE ÚNICO U OBRAS DE ARTE',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR COMERCIALIZACIÓN DIRECTA',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS POR MANTENIMIENTO SIN PRECISIÓN DEL ALCANCE EN LOS TRABAJOS',
'INVITACIÓN A CUANDO MENOS 3 PERSONAS EN LOS TRABAJOS']

In [5]:
def filtro_procedimientos(lista, nuevo):
    for valor in contratos_2024['Tipo Procedimiento']:
        if valor in lista:
            contratos_2024['Tipo Procedimiento'] = contratos_2024['Tipo Procedimiento'].str.replace(pat=valor,
                                                                                                    repl=nuevo,
                                                                                                    regex=False)

In [6]:
filtro_procedimientos(adjudicaciones, 'ADJUDICACIÓN DIRECTA')
filtro_procedimientos(invitaciones, 'INVITACIÓN A CUANDO MENOS 3 PERSONAS')

### Hallazgos
Este análisis tiene como objetivo echar a andar una investigación periodística. No se puede hablar de conclusiones como tal. Sin embargo, podemos señalar varios hallazgos que podrían abrir brecha para iniciar indagaciones de interés.

#### Los más beneficiados
Las instituciones que más dinero gastaron en contratos públicos durante 2024 son:
1. IMSS→ MXN 119,276 millones
2. ISSSTE→ MXN 24,327 millones
3. SEDENA → MXN 20,474 millones
4. IMSS BIENESTAR → MXN 18,503 millones
5. DICONSA →  MXN 13,987 millones



In [8]:
mayores_compradores = contratos_2024.groupby('Institución').agg({'Importe DRC':'sum'})
mayores_compradores.columns = ['Importe DRC Total']
mayores_compradores = mayores_compradores.reset_index()
mayores_compradores = mayores_compradores.sort_values(by='Importe DRC Total', ascending=False)
mayores_compradores.head()

,Institución,Importe DRC Total
328,INSTITUTO MEXICANO DEL SEGURO SOCIAL,119276637897
301,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,24327146787
541,SECRETARIA DE LA DEFENSA NACIONAL,20474194210
621,SERVICIOS DE SALUD DEL INSTITUTO MEXICANO DEL ...,18503718837
199,"DICONSA, S.A. DE C.V.",13987043427


Los contratos más jugosos fueron otorgados por:
1. Hacienda → Agroasemex, procedimiento entre entes públicos (MXN 5,295 millones)
2. Bienestar → Integra Arrenda, adjudicación directa (MXN 3,641 millones)
3. Fonatur/Tren Maya →Operadora CICSA, adjudicación directa (MXN 3,352 millones)
4. IMSS Bienestar → TOKA Internacional, licitación pública (MXN 2,694 millones)
5. Guardia Nacional →Edenred México, licitación pública (MXN 2,598 millones)

In [9]:
contratos_caros = contratos_2024.sort_values(by='Importe DRC', ascending=False) 
contratos_caros.head()

,Descripción Ramo,Institución,Nombre de la UC,Título del contrato,Descripción del contrato,Importe DRC,Proveedor o contratista,rfc,Nacionalidad proveedor o contratista,Estratificación,Tipo Procedimiento,Número de procedimiento,Código del contrato
4036,HACIENDA Y CRÉDITO PÚBLICO,SECRETARIA DE HACIENDA Y CREDITO PUBLICO,COORDINACIÓN DE ADQUISICIONES Y CONTRATACIÓN D...,SEGURO PARAMÉTRICO DEL RIESGO CATASTRÓFICO DEL...,SEGURO PARAMÉTRICO RELACIONADO CON LA IMPLEMEN...,5295180017,AGROASEMEX SA,AGR900605VC6,MEXICANA,NO MIPYME,ENTRE ENTES PÚBLICOS,AA-06-712-006000998-N-55-2024,C-2024-00024748
780,BIENESTAR,SECRETARIA DE BIENESTAR,DIRECCION GENERAL DE RECURSOS MATERIALES,SERVICIO DE ARRENDAMIENTO TRANSPORTE VEHICULAR...,SERVICIO DE ARRENDAMIENTO TRANSPORTE VEHICULAR...,3641577670,INTEGRA ARRENDA SA DE CV SOFOM ENR,AAN910409I35,MEXICANA,GRANDE,ADJUDICACIÓN DIRECTA,AA-20-411-020000999-N-48-2024,C-2025-00006673
97893,TURISMO,"FONATUR TREN MAYA, S.A. DE C.V.",SUBDIRECCION DE ESCALAS NAUTICAS,TRABAJOS EXTRAORDINARIOS NO CONSIDERADOS PARA ...,FONATUR TREN MAYA RECONOCE LOS TRABAJOS EXTRAO...,3352325414,OPERADORA CICSA SA DE CV,OCI810921EI3,MEXICANA,GRANDE,ADJUDICACIÓN DIRECTA,AO-21-W3X-021W3X999-N-28-2024,C-2024-00146901
131325,ENTIDADES NO SECTORIZADAS,SERVICIOS DE SALUD DEL INSTITUTO MEXICANO DEL ...,COORDINACIÓN DE ADQUISICIONES,SERVICIO DE SUMINISTRO DE VALES ELECTRÓNICOS M...,SERVICIO DE SUMINISTRO DE VALES ELECTRÓNICOS M...,2694738000,TOKA INTERNACIONAL S A P I DE CV,TIN090211JC9,MEXICANA,GRANDE,LICITACIÓN PÚBLICA,LA-27-703-027703982-N-23-2024,C-2024-00147899
8831,SEGURIDAD Y PROTECCIÓN CIUDADANA,GUARDIA NACIONAL,DIRECCION GENERAL DE RECURSOS MATERIALES,SERVICIO DE SUMINISTRO DE COMBUSTIBLE PARA VEH...,SERVICIO DE SUMINISTRO DE COMBUSTIBLE PARA VEH...,2598842776,EDENRED MEXICO SA DE CV,ASE930924SS7,MEXICANA,NO MIPYME,LICITACIÓN PÚBLICA,LA-27-703-027703982-N-9-2024,C-2023-00172407


Los proveedores que recibieron la mayor cantidad de contratos en 2024 fueron:
1. Molinos Azteca → 1191
2. Alen del Norte → 967
3. Dagmar → 923
4. Brand and Push → 871
5. Liconsa → 840

In [10]:
proveedores_beneficiados = contratos_2024[['Proveedor o contratista', 'Importe DRC']]
proveedores_beneficiados = proveedores_beneficiados.groupby('Proveedor o contratista').agg({'Proveedor o contratista':'count'})
proveedores_beneficiados.columns = ['Conteo']
proveedores_beneficiados = proveedores_beneficiados.reset_index()
proveedores_beneficiados.sort_values(by='Conteo', ascending=False)

,Proveedor o contratista,Conteo
27847,MOLINOS AZTECA SA DE CV,1191
1523,ALEN DEL NORTE SA DE CV,967
14144,GADMAR SA DE CV,923
4672,BRAND AND PUSH SA DE CV,871
23343,LICONSA SA DE CV,840
...,...,...
18757,IZALIA GUADALUPE CABRERA RODRIGUEZ,1
18756,IZAI LOPEZ MACIAS,1
18755,IZAAC EZEQUIEL CASILLAS ANGUIANO,1
18754,IYELITZIN CASTRO FRANCO,1


#### Dominan las adjudicaciones directas
La mayor parte (78%) del dinero otorgado en contratos públicos durante 2024 se hizo mediante adjudicaciones directas. El 14% del monto total correspondió a licitaciones públicas, el 5% a invitaciones de cuando menos tres personas y el 1.3% a procedimientos entre entes públicos.


In [11]:
tipos_de_contrato= contratos_2024.groupby(['Tipo Procedimiento']).agg({'Tipo Procedimiento':'count'})
tipos_de_contrato.columns= ['Conteo']
tipos_de_contrato = tipos_de_contrato.reset_index()
tipos_de_contrato = tipos_de_contrato.sort_values(by='Conteo', ascending=False).head()
tipos_de_contrato

,Tipo Procedimiento,Conteo
0,ADJUDICACIÓN DIRECTA,109205
3,LICITACIÓN PÚBLICA,20490
2,INVITACIÓN A CUANDO MENOS 3 PERSONAS,7042
1,ENTRE ENTES PÚBLICOS,1747
4,SE AGREGA PARA VINCULAR CON MFIJ,11


En promedio, cada institución de gobierno otorgó 45% del dinero total por contratos en 2024 en contratos por adjudicación directa. Algunas instituciones que superaron ese promedio son:
* IMSS → 49%
* ISSSTE → 65%
* DICONSA → 68%
* Secretaría de Bienestar → 73$
* Fonatur/Tren Maya → 88%
* SEMAR → 57%
* Ferrocarril del Istmo de Tehuantepec → 90%
* Administración del Sistema Portuario Nacional Dos Bocas → 99%
* Banco del Bienestar → 70%
* Tribunal Federal de Justicia Administrativa → 56%
* Secretaría Anticorrupción y Buen Gobierno → 78%
* Secretaría de Infraestructura →94%
* Secretaría de Comunicaciones y Obras Públicas →96%
* Bancomext → 79%
* Canal 11 → 96%

In [13]:
instituciones_procedimientos = contratos_2024.groupby(['Institución', 'Tipo Procedimiento']).agg({'Importe DRC':'sum'})
instituciones_procedimientos = instituciones_procedimientos.rename({'Importe DRC':'Importe Total Procedimiento'}, axis=1)
instituciones_procedimientos = instituciones_procedimientos.reset_index()

adjudicaciones_directas = instituciones_procedimientos[instituciones_procedimientos['Tipo Procedimiento'] == 'ADJUDICACIÓN DIRECTA']
adjudicaciones_directas = adjudicaciones_directas.sort_values(by='Importe Total Procedimiento', ascending=False)

contratos_merged = pd.merge(left=mayores_compradores,
                            right=adjudicaciones_directas,
                            left_on='Institución',
                            right_on='Institución',
                            how='left')

contratos_merged['Porcentaje Adjudicaciones'] = [contratos_merged['Importe Total Procedimiento'].loc[row] / contratos_merged['Importe DRC Total'].loc[row] *100 for row in range(len(contratos_merged['Institución']))] 

mask_contratos = contratos_merged['Porcentaje Adjudicaciones'] > 45
mask2_contratos = contratos_merged['Porcentaje Adjudicaciones'] != 100
adjudicaciones_mask = contratos_merged[mask_contratos & mask2_contratos].reset_index()
adjudicaciones_mask

,index,Institución,Importe DRC Total,Tipo Procedimiento,Importe Total Procedimiento,Porcentaje Adjudicaciones
0,0,INSTITUTO MEXICANO DEL SEGURO SOCIAL,119276637897,ADJUDICACIÓN DIRECTA,58640874533,49
1,1,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,24327146787,ADJUDICACIÓN DIRECTA,15933591794,65
2,4,"DICONSA, S.A. DE C.V.",13987043427,ADJUDICACIÓN DIRECTA,9572216332,68
3,11,SECRETARIA DE BIENESTAR,5288753569,ADJUDICACIÓN DIRECTA,3842584706,73
4,13,"FONATUR TREN MAYA, S.A. DE C.V.",4397366638,ADJUDICACIÓN DIRECTA,3859171482,88
...,...,...,...,...,...,...
134,647,UNIVERSIDAD TECNOLÓGICA DE TECAMACHALCO,1892516,ADJUDICACIÓN DIRECTA,1608033,85
135,650,EMILIANO ZAPATA,1809316,ADJUDICACIÓN DIRECTA,872075,48
136,659,UNIVERSIDAD TECNOLÓGICA BILINGÜE INTERNACIONAL...,1589564,ADJUDICACIÓN DIRECTA,1344254,85
137,731,COLEGIO DE ESTUDIOS CIENTÍFICOS Y TECNOLÓGICOS...,466328,ADJUDICACIÓN DIRECTA,334400,72


#### Sedena
Debido a la opacidad que rodea al ejército mexicano, se consideró pertinente echar un vistazo a los contratos otorgados por la Secretaría de la Defensa Nacional (Sedena) este año. 

Consideramos destacar la lista de “proveedores favoritos” de Sedena. Abajo se encuentran los que recibieron la mayor cantidad de dinero en contratos durante el año. 
1. ANGAR AZCAPOTZALCO SA DE CV → MXN 25,151 millones
2. TRANS CE CARGO SA DE CV → MXN 22, 531 millones
3. CAMARA NACIONAL DE AUTOTRANSPORTE DE PASAJE Y TURISMO → MXN 7,939 millones
4. CONTROLADORA MABE → MXN 6,482 millones
5. LIAD PRODUCTOS PARA ROPA Y CALZADO SA DE CV → MXN 5,913 millones

In [14]:
sedena = contratos_2024.loc[contratos_2024['Descripción Ramo'] == 'DEFENSA NACIONAL']

sedena_proveedores = sedena.groupby(['Institución', 'Proveedor o contratista']).agg({'Importe DRC':'sum'}).sort_values(by='Importe DRC', ascending=False)
sedena_proveedores.head()

Importe DRC
Institución                       Proveedor o contratista                                        
SECRETARIA DE LA DEFENSA NACIONAL ANGAR AZCAPOTZALCO SA DE CV                          2515115550
                                  TRANS CE CARGO S DE RL DE CV                         2253100000
                                  CAMARA NACIONAL DEL AUTOTRANSPORTE DE PASAJE Y ...    793965517
                                  CONTROLADORA MABE SA DE CV                            648200000
                                  LIAD PRODUCTOS PARA ROPA Y CALZADO SA DE CV           591307740

Los proveedores con la mayor cantidad de contratos de Sedena en 2024 fueron:
1. HWG Process and Solutions → 60
2. Distribuciones Orval → 46
3. Comercializadora Mexicana GC2 → 40
4. Distribuidora Yovic → 29
5. Distribuidora Lareal Olivera y Lopez → 27

In [15]:
sedena_contratos = sedena.groupby(['Institución', 'Proveedor o contratista']).agg({'Proveedor o contratista':'count'})
sedena_contratos.columns = ['Conteo']
sedena_contratos = sedena_contratos.reset_index()
sedena_contratos.sort_values(by='Conteo', ascending=False)

,Institución,Proveedor o contratista,Conteo
977,SECRETARIA DE LA DEFENSA NACIONAL,HGW PROCESS AND SOLUTIONS SA DE CV,60
758,SECRETARIA DE LA DEFENSA NACIONAL,DISTRIBUCIONES ORVAL SA DE CV,46
641,SECRETARIA DE LA DEFENSA NACIONAL,COMERCIALIZADORA MEXICANA GC2 SA DE CV,40
781,SECRETARIA DE LA DEFENSA NACIONAL,DISTRIBUIDORA YOVIC SA DE CV,29
775,SECRETARIA DE LA DEFENSA NACIONAL,DISTRIBUIDORA LAREAL OLIVERA Y LOPEZ SA DE CV,27
...,...,...,...
557,SECRETARIA DE LA DEFENSA NACIONAL,CALZADO LEED DE MEXICO SA DE CV,1
556,SECRETARIA DE LA DEFENSA NACIONAL,"CALIDAD, SALUD Y DIAGNOSTICO SA DE CV",1
550,SECRETARIA DE LA DEFENSA NACIONAL,BUFETE DE INFORMATICA COMPUTACIONAL SA DE CV,1
548,SECRETARIA DE LA DEFENSA NACIONAL,BRUNTON ALLIANCE SA DE CV,1


Varios de estos proveedores (si no es que todos) ameritan una mirada más cuidadosa e indagaciones más profundas. El compadrismo y el nepotismo son comunes en México cuando de licitaciones públicas se trata. Cabe la posibilidad de que por lo menos unos cuantos de los proveedores más beneficiados por estas licitaciones a lo largo de 2024 y años anteriores sean demasiado cercanos a los funcionarios que controlan los procesos de licitación (directa o indirectamente).

Recomiendo comenzar con Molinos Azteca (un proveedor importante de Diconsa) y Angar Azcapotzalco (proveedor preferido de la Sedena). Sobre este último [ya se escribieron](https://www.debate.com.mx/politica/Sedena-otorga-contratos-a-Grupo-Andrade-por-mas-de-8-mil-millones-de-pesos-20230216-0092.html) varios artículos periodísticos señalando su conexión con Grupo Andrade, empresa dueña de El Heraldo de México.